In [13]:
import sys
import numpy as np
import pandas as pd
from IPython.display import display

".." in sys.path or sys.path.append("..")
from alph import alph

In [27]:
# Ref: https://openflights.org/data.html

routes_url = "https://raw.githubusercontent.com/jpatokal/openflights/master/data/routes.dat"
airports_url = "https://raw.githubusercontent.com/jpatokal/openflights/master/data/airports.dat"

airports_cols = [
    x.lower().replace(" ", "_")
    for x in [
        "Airport ID", "Name", "City", "Country", "IATA",
        "ICAO", "Latitude", "Longitude", "Altitude", 
        "Timezone", "DST", "Tz database time zone", "Type", "Source",
    ]
]
routes_cols = [
    x.lower().replace(" ", "_")
    for x in [
        "Airline", "Airline ID",
        "Source airport", "Source airport ID",
        "Destination airport", "Destination airport ID",
        "Codeshare", "Stops", "Equipment"
    ]
]

airports = pd.read_csv(airports_url, names=airports_cols)
routes = pd.read_csv(routes_url, names=routes_cols)

print(airports.shape[0], routes.shape[0])
display(routes.head(2))
display(airports.head(2))

7698 67663


,airline,airline_id,source_airport,source_airport_id,destination_airport,destination_airport_id,codeshare,stops,equipment
0,2B,410,AER,2965,KZN,2990,NaN,0,CR2
1,2B,410,ASF,2966,KZN,2990,NaN,0,CR2


,airport_id,name,city,country,iata,icao,latitude,longitude,altitude,timezone,dst,tz_database_time_zone,type,source
0,1,Goroka Airport,Goroka,Papua New Guinea,GKA,AYGA,-6.08169,145.391998,5282,10,U,Pacific/Port_Moresby,airport,OurAirports
1,2,Madang Airport,Madang,Papua New Guinea,MAG,AYMD,-5.20708,145.789001,20,10,U,Pacific/Port_Moresby,airport,OurAirports


In [30]:
# routes.pipe(
#     #lambda d: d[d["stops"] == 0]
# )
routes.assign(
    a_id=lambda d: np.where(
        d["source_airport_id"] <= d["destination_airport_id"],
        d["source_airport_id"],
        d["destination_airport_id"]
    ),
    b_id=lambda d: np.where(
        d["source_airport_id"] <= d["destination_airport_id"],
        d["destination_airport_id"],
        d["source_airport_id"]
    ),
).pipe(
    lambda d: d[d["a_id"] != d["b_id"]]
).groupby(
    ["a_id", "b_id"]
)["airline_id"].count().sort_values(ascending=False)

a_id  b_id
3682  3830    39
3077  3885    24
3797  507     24
3576  3682    24
3179  3885    23
              ..
5419  5979     1
2960  3941     1
1203  1208     1
5431  5433     1
1759  1761     1
Name: airline_id, Length: 19203, dtype: int64